In [9]:
import pandas as pd
recipe_mpr = pd.read_json('https://raw.githubusercontent.com/D3Mlab/Recipe-MPR/main/data/500QA.json', orient = 'records')
recipe_mpr

,query,query_type,options,answer,correctness_explanation
0,I want to make a warm dish containing oysters,"{'Specific': 0, 'Commonsense': 1, 'Negated': 0...","{'08cb462fdf': 'Simple creamy oyster soup', '5...",08cb462fdf,"{'warm dish': 'soup', 'oysters': 'oyster'}"
1,Can I have a recipe for fish that's roasted?,"{'Specific': 0, 'Commonsense': 0, 'Negated': 0...","{'069aa1f8af': 'Salmon roasted with olive oil,...",069aa1f8af,"{'fish': 'Salmon', 'roasted': 'roasted'}"
2,"What are recipes for fish, but not baked in th...","{'Specific': 0, 'Commonsense': 1, 'Negated': 1...",{'3536b10328': 'Salmon fillets grilled in a pa...,3536b10328,"{'fish': 'Salmon', 'not baked in the oven': 'g..."
3,I would like a shrimp recipe and I'm trying t...,"{'Specific': 0, 'Commonsense': 1, 'Negated': 0...",{'00a39a8659': 'Shrimp roasted with vegetables...,00a39a8659,"{'shrimp': 'shrimp', 'balanced diet': 'with ve..."
4,Can I have a shrimp pasta recipe for someone w...,"{'Specific': 1, 'Commonsense': 1, 'Negated': 0...",{'3c9d5c4033': 'Shrimp and vegetables with spa...,3c9d5c4033,"{'shrimp': 'Shrimp', 'pasta': 'spaghetti', 'lo..."
...,...,...,...,...,...
495,"I have to use some eggs before they get bad, w...","{'Specific': 0, 'Commonsense': 1, 'Negated': 0...",{'14cf9f5a09': 'Creamy base with dill flavorin...,14cf9f5a09,"{'eggs': 'eggs', 'noodle dish': 'noodles'}"
496,What is a side dish recipe that will set my mo...,"{'Specific': 0, 'Commonsense': 1, 'Negated': 0...",{'1158f0fbd4': 'Assorted vegetables with cayen...,1158f0fbd4,"{'side dish': 'assorted vegetables', 'set my m..."
497,What's a spicy dish for someone who has lower ...,"{'Specific': 0, 'Commonsense': 1, 'Negated': 0...","{'bb089cf062': 'Mildly spicy chili recipe', '2...",bb089cf062,"{'spicy': ['spicy', 'chili'], 'lower spice tol..."
498,Whats a marinade I could make to make my ribey...,"{'Specific': 0, 'Commonsense': 0, 'Negated': 0...","{'298b55bc10': 'Ribeye steak marinade', '1bb23...",298b55bc10,"{'ribeye': 'ribeye', 'marinade': 'marinade', '..."


In [2]:
recipe_mpr_clean = (
    recipe_mpr
    .assign(
        query_aspects = lambda d: d.correctness_explanation.apply(lambda x: list(x.keys())),
        num_aspects = lambda d: d.query_aspects.apply(len),
        option_id = lambda d: d.options.apply(lambda x: list(x.keys())),
        option_text = lambda d: d.options.apply(lambda x: list(x.values()))
    )
    .query("num_aspects > 1")
    .explode(['option_id', 'option_text'])
    [['query', 'option_id', 'option_text']]
)

recipe_mpr_clean

,query,option_id,option_text
0,I want to make a warm dish containing oysters,08cb462fdf,Simple creamy oyster soup
0,I want to make a warm dish containing oysters,5b9441298f,Seasoned salted crackers shaped like oysters
0,I want to make a warm dish containing oysters,00310c3462,Creamy clam chowder made with whole milk and b...
0,I want to make a warm dish containing oysters,8635ea3d3c,Tomato mussel soup containing dry white wine
0,I want to make a warm dish containing oysters,52b83497d8,"Warm vegetable soup containing tomatoes, peas,..."
...,...,...,...
499,What's a good broccoli side dish I can make fo...,aab9dd06db,Broccoli salad with bacon
499,What's a good broccoli side dish I can make fo...,54a96ce8e4,Oven cooked chicken meat seasoned with paprika
499,What's a good broccoli side dish I can make fo...,1d5f95763f,Broccoli and onions cooked with red wine
499,What's a good broccoli side dish I can make fo...,29500eb70f,Beef paired with bacon entree


In [54]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-tas-b')

queries = []
items = []
scores = []

for name, group in recipe_mpr_clean.groupby('query'):
    queries.append(name)
    items.append(list(group['option_text']))
    query_emb = model.encode(name)
    item_embs = model.encode(list(group['option_text']))
    out = util.dot_score(query_emb, item_embs)
    scores.append(out.detach().numpy()[0])

import numpy as np
scores_df = (
    pd.DataFrame({'query': queries, 'score': scores, 'option_text': items})
    .explode(["score", "option_text"])
)

scores_df

,query,score,option_text
0,I would like a shrimp recipe and I'm trying t...,92.793968,Shrimp roasted with vegetables in a zesty Ital...
0,I would like a shrimp recipe and I'm trying t...,90.752785,Shrimp balls made with pork fat deep fried in oil
0,I would like a shrimp recipe and I'm trying t...,90.600403,Breaded shrimp with whole milk and creole seas...
0,I would like a shrimp recipe and I'm trying t...,85.023895,Cucumber sauce with fat-free yogurt and reduce...
0,I would like a shrimp recipe and I'm trying t...,84.706909,Healthy napa cabbage and green onion coleslaw ...
...,...,...,...
499,Would I be able to get a breakfast recipe with...,87.291847,Fish meat cooked on a grill / stovetop / oven
499,Would I be able to get a breakfast recipe with...,86.487907,Fish meat cooked in a steamer
499,Would I be able to get a breakfast recipe with...,86.148117,Fish meat prepared with swiss cheese
499,Would I be able to get a breakfast recipe with...,87.565262,Seafood linguine recipe containing shrimp


In [56]:
scores_df.to_csv("query_dense_scores.csv")